In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt

def rough_plot(in_list, num_group):
    rough_list = list()
    tmp_sum = 0
    
    for i in range(1, len(in_list)):
        tmp_sum += in_list[i]
    
        if(i%num_group == num_group-1):
            rough_list.append(tmp_sum / num_group)
            tmp_sum = 0
            
    plt.plot(rough_list)

In [3]:
import sys

def next_state(state, action, action_level):
    next_state = state.copy()
    if (action > 2*action_level) or action < 0:
        print("action = ", action)
        sys.exit("error, action undefined! nxt")

    if action >= 1 and action <= action_level:
        # then buy
        
        if state['HandCash'] > 0:
            next_state['HandCash'] = state['HandCash']*(1 - action/action_level)
            next_state['NumStock'] = state['NumStock'] + \
                                     (state['HandCash']/state['StockPrice'])*(action/action_level)

    elif action >= (action_level + 1) and action <= (action_level + action_level) :
        # then sell
        
        if state['NumStock'] > 0:
            next_state['NumStock'] = state['NumStock']*(1 - (action-action_level)/action_level)
            next_state['HandCash'] = state['HandCash'] + \
                                     (state['StockPrice']*state['NumStock'])*((action-action_level)/action_level)
    
    return next_state


def reward(state, init_invest):
    return (state['NumStock']*state['StockPrice'] + state['HandCash']) - init_invest

def my_reward(curr, last):
    return (curr['NumStock']*curr['StockPrice'] + curr['HandCash']) - \
           (last['NumStock']*last['StockPrice'] + last['HandCash'])

In [4]:
from random import randint
import math

relist = ['QCOM.csv', 'GOOG.csv', 'NOK.csv', '^DJI.csv', '^TWII.csv', '^N225.csv', '^GDAXI.csv', \
          '^IXIC.csv', '^KS11.csv', '^SSEC.csv', \
         'NSU.DE_10_5_5.csv', 'DAI.DE_10_5_5.csv', 'BMW.DE_10_5_5.csv']

# for data in input/2001_2010/
relist = ['QCOM.csv', 'NOK.csv', '^DJI.csv', '^TWII.csv', '^N225.csv', '^GDAXI.csv', \
          '^IXIC.csv', '^KS11.csv', '^SSEC.csv', \
         'NSU.DE.csv', 'DAI.DE.csv', 'BMW.DE.csv']

#relist = ['^IXIC.csv']

last_rwd_func = reward
avg_point_rwd_func = my_reward

action_level = 3
init_invest = 10000
trial_range = 1000

for resource_data in relist:
    
    #input_csv = "../input/{0}".format(resource_data)
    input_csv = "../input/2001_2010/{0}".format(resource_data)
    qcom = pd.read_csv(input_csv)
    qclose = qcom['Close']
    qclose = qclose.fillna(method='ffill')
    qclose = qclose.tolist()
    np.isnan(qclose).any()


    last_reward_list = list()


    for trial in range(0, trial_range):

        state = {'NumStock': 0, 'StockPrice': qclose[0], 'HandCash': init_invest}

        for point in range(0, len(qclose)):

            # new price revealed!
            state['StockPrice'] = qclose[point]

            if point == len(qclose)-1:
                last_reward_list.append(last_rwd_func(state, init_invest))


            # determine the next state
            act = randint(0, 2*action_level)
            state = next_state(state, act, action_level)

    print(resource_data, sum(last_reward_list) / float(len(last_reward_list)))

QCOM.csv 4362.479009836666
NOK.csv -3544.0370194918905
^DJI.csv 870.0999905823795
^TWII.csv 4379.444919644278
^N225.csv -647.5721320195088
^GDAXI.csv 1567.9605517373438
^IXIC.csv 1416.1912325075104
^KS11.csv 11651.006624929983
^SSEC.csv 2739.8850994019
NSU.DE.csv 36198.77458528407
DAI.DE.csv 2366.39508434821
BMW.DE.csv 5257.5096434246125
